# Figure Out Merging

In [1]:
# Imports
import os
import sys

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from dask import delayed, compute
from dask.diagnostics import ProgressBar

# Project level imports
sys.path.insert(0, '../../lib/python')
from ncbi_remap.notebook import Nb
from ncbi_remap.parser import parse_picardCollect_summary
from ncbi_remap.io import add_table, add_id, remove_id
from ncbi_remap.snakemake import combine

# Setup notebook
nbconfig = Nb.setup_notebook()

# Turn on cache
from joblib import Memory
memory = Memory(cachedir=nbconfig.cache, verbose=0)

last updated: 2017-11-02 
Git hash: 77256d4c91158a79506f4adb8e290ae30d5f9ec0


In [2]:
store_old = pd.HDFStore('../../sra.h5')

In [3]:
store_new = pd.HDFStore('../../sra.comp.h5')

In [8]:
store_old

<class 'pandas.io.pytables.HDFStore'>
File path: ../../sra.h5
/aln/alignment_bad                                          frame_table  (typ->appendable,nrows->84,ncols->2,indexers->[index],dc->[srr,srx])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
/aln/complete                                               frame_table  (typ->appendable,nrows->15157,ncols->2,indexers->[index],dc->[srr,srx])                                                                                       

In [9]:
store_new

<class 'pandas.io.pytables.HDFStore'>
File path: ../../sra.comp.h5
/aln/alignment_bad                                          frame_table  (typ->appendable,nrows->84,ncols->2,indexers->[index],dc->[srr,srx])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
/aln/complete                                               frame_table  (typ->appendable,nrows->15157,ncols->2,indexers->[index],dc->[srr,srx])                                                                                  

In [11]:
store_new['prealn/workflow/hisat2/PE']

,srx,srr,num_reads,num_reads_paired,num_concordant_reads_unaligned,num_concordant_reads_uniquely_aligned,num_concordant_multimappers,num_discordant_reads_aligned,num_unaligned,num_uniquely_aligned,num_multimappers,per_alignment
0,SRX2581987,SRR5278103,17428278,17428278,15689049,1344526,394703,18408,30692893,497323,151066,11.95
1,SRX2540527,SRR5232052,53756361,53756361,9403758,37596048,6756555,2880864,10869312,1327721,848755,89.89
2,SRX2540526,SRR5232051,46842643,46842643,8847919,32229178,5765546,2933174,9350828,1548605,930057,90.02
3,SRX2540525,SRR5232050,53670986,53670986,9490455,37374453,6806078,2923263,10913971,1384521,835892,89.83
4,SRX2540524,SRR5232049,44580338,44580338,7163393,31424379,5992566,2146018,7787456,1440854,806440,91.27
5,SRX2540523,SRR5232048,51853737,51853737,10083893,34866461,6903383,3314495,10562316,1835636,1140844,89.82
6,SRX2540522,SRR5232047,46660898,46660898,8802070,31619372,6239456,2966465,8798862,1779639,1092709,90.57
7,SRX2531409,SRR5221633,85735654,85735654,14872300,51593656,19269698,1550715,22363209,2724994,1554967,86.96
8,SRX2531408,SRR5221632,66998251,66998251,10744682,40052597,16200972,1224870,15467310,2244179,1328135,88.46
9,SRX2531407,SRR5221631,84928778,84928778,13367845,51286960,20273973,1729134,19207850,2452893,1616679,88.69


In [13]:
store_old.close()
store_new.close()